In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr

In [2]:
# DB = pd.read_csv("D:/实验室/数据/new_eicuDataV1.0.csv").drop(columns=["st1","creatinine"],axis =1)
DB = pd.read_csv("D:/实验室/数据/eicu数据处理/Imp6_cart.csv")
Feature_map = pd.read_excel("D:/实验室/数据/0909特征范围.xlsx")
Feature_map = Feature_map.drop(columns="no",axis = 1)

DB["sex"] = DB["sex"]-1

In [3]:
#删除-1的label,age和sex的特征的样本
DB = DB[DB["label"]>=0]
DB = DB.drop(DB[(DB["sex"].isna())+(DB["sex"]=="Other")+(DB["sex"]=="Unknown")].index)
DB = DB.drop(index = DB[DB["age"]<18].index,axis = 1)
DB = DB.drop(index = DB[DB["age"].isna()].index)
#处理race将other什么的转换为nan
DB.loc[DB["race"]=="Other/Unknown","race"]=np.nan
#性别转换
DB.loc[DB["sex"]=="Female","sex"]=0
DB.loc[DB["sex"]=="Male","sex"]=1
#种族转换
DB.loc[DB["race"]=="Caucasian","race"]=1
DB.loc[DB["race"]=="African American","race"]=2
DB.loc[DB["race"]=="Hispanic","race"]=3
DB.loc[DB["race"]=="Asian","race"]=4
DB.loc[DB["race"]=="Native American","race"]=5
#年龄处理
DB.loc[DB["age"]=="> 89","age"]=90
DB.loc[~DB["age"].isnull(),"age" ]= DB[~DB["age"].isnull()]["age"].astype(int)

#将数据全部转化为浮点型
DB = DB.astype(np.float16)
#3sigma原则处理异常值
def three_sigma(ser):
 
    bool_id = ~((ser.mean() - 3 * ser.std()) <= ser)  &   (ser <= (ser.mean() + 3 * ser.std()))
    
    
    return bool_id
sigma_list = list(DB.columns)[5:77]
# for col in sigma_list:
#     DB.loc[three_sigma(DB[col]),col]=np.nan
    
    
DB_1 = DB.copy(deep = True)    
Discrete_name = ["age","bmi","SBP","DBP"]
Discrete_name = ["age","bmi","SBP","DBP"]
 
Discrete_index = [[16,25,35,45,55,65,75],[18.5,25,30],[120,140,150,160,180],[80,90,100,110]]
for Discrete,col in zip(Discrete_index,Discrete_name):
    Discrete_len = len(Discrete)+1
    if col in DB_1.columns:
        for i in range(Discrete_len):
            if i==0:
                DB_1.loc[DB[col]<Discrete[i],col]=i+1
            elif i<Discrete_len-1:
                DB_1.loc[(DB[col]>=Discrete[i-1])&(DB[col]<Discrete[i]),col]=i+1
            elif i==Discrete_len-1:
                DB_1.loc[DB[col]>=Discrete[i-1],col]=i+1
       
# Discrete_name_MF = ["rbc","ferritin","cpk","st2","hct","hgb"]
Discrete_name_MF = ["LAB6","LAB22","LAB26","LAB31","LAB43","LAB44"]
Discrete_index_MF = [([4.7,6.1],[4.2,5.4]),([12,300],[12,150]),([39,308],[26,192]),([86,49.3],[7.2,33.5]),([41,45],[36,38]),([13.2,17.2],[12.1,15.1])]
for col,index in zip(Discrete_name_MF,Discrete_index_MF):
    if col in DB_1.columns:
        for sex in range(2):
    
            DB_1.loc[(DB["sex"]==sex)&(DB[col]<index[sex][0]),col] = 1
            DB_1.loc[(DB["sex"]==sex)&(DB[col]>=index[sex][0])&(DB[col]<index[sex][1]),col] = 2
            DB_1.loc[(DB["sex"]==sex)&(DB[col]>=index[sex][1]),col] = 3

# drop_name = ["age","bmi","SBP","DBP","race","sex","rbc","ferritin","st2","hct","hgb","st1"]
drop_name = ["age","bmi","DBP","race","sex","LAB6","LAB22","LAB31","LAB43","LAB44","LAB30"]

for i in range(75):
    #获取特征名字
    name = Feature_map.iloc[i,1]
    #获取特征下界限
    if name in DB_1.columns:
        if name not in drop_name:

            low=Feature_map.iloc[i,2]
            up=Feature_map.iloc[i,3]

            DB_1.loc[DB[name]<low,name]=1
            DB_1.loc[(DB[name]>=low)&(DB[name]<up),name]=2
            DB_1.loc[DB[name]>=up,name]=3


In [4]:
DB_1.to_csv("D:/实验室/数据/eicu数据处理/Imp6_cart_2.0.csv")

In [5]:
for name in Feature_map["name"]:
    if name in DB.columns:
        low = float(Feature_map[Feature_map["name"]==name]["low"])
        if low ==999:
            pass
        else:
            low=float(Feature_map[Feature_map["name"]==name]["low"])
            up=float(Feature_map[Feature_map["name"]==name]["up"])
            DB_1.loc[DB[name]<low,name]=1
            DB_1.loc[(DB[name]>=low)&(DB[name]<up),name]=2
            DB_1.loc[DB[name]>=up,name]=3
            


In [6]:
#删除特征缺失率较高特征
for col in DB_1:
    loss_rate = sum(DB_1[col].isnull())/len(DB_1)*100
    if loss_rate > 75:
        DB_1= DB_1.drop(columns = col , axis = 1)
DB_1 = DB_1.iloc[:,:63]

In [7]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix
def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix[col][index]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_matrix(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_matrix(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)    
def print_dag(dag):
    for edges in dag.edges:
        print(edges[0],"->",edges[1],";")

In [8]:
class client():
    
    def __init__(self,data,cid,black):
        self.black_list = black
        self.data = data
        self.cid = cid
        self.model = HillClimbSearch(data)
    def get_model(self):
        #获得本地客户端网络结构
        return self.best_model
    def fit(self):
        #记录最好的模型
        
        scoring_method = K2Score(data=self.data)
        self.best_model = self.model.estimate(black_list = self.black_list)
        return 1
    def set_parment(self):
        #从服务器中心get到网络结构
        #初始化model
        model.edges
    
        return 
    def update_model(self,fuse_dag):
        print("开始更新")
        self.model = HillClimbSearch(self.data)
        scoring_method = K2Score(data=self.data)
#         self.best_model = self.model.estimate(scoring_method=scoring_method,white_list=fuse_dag.edges,black_list = self.black_list)
        self.best_model = self.model.estimate(fixed_edges=fuse_dag.edges,black_list = self.black_list)
        
    

In [9]:
class client_R():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid,black):
        #black 用R语言的存储方式存储·
        self.black_list = black
        self.data = data
        self.cid = cid
        #初始化创建一个空的图
        self.model = DAG()
    def get_model(self):
        #获得本地客户端网络结构
        return self.best_model
    def fit(self):
        #记录最好的模型
        # 代改动部分        
        scoring_method = K2Score(data=self.data)
        self.best_model = self.model.estimate(black_list = self.black_list)
        
    
        return 1
    def set_parment(self):
        #从服务器中心get到网络结构
        #初始化model
        model.edges
    
        return 
    def update_model(self,fuse_dag):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
        print("开始更新")
        self.model = HillClimbSearch(self.data)
        scoring_method = K2Score(data=self.data)
#         self.best_model = self.model.estimate(scoring_method=scoring_method,white_list=fuse_dag.edges,black_list = self.black_list)
        self.best_model = self.model.estimate(fixed_edges=fuse_dag.edges,black_list = self.black_list)

In [10]:
class sever():
    def __init__(self,fuse_num):
        self.fuse_num = fuse_num
    def get_edges_matrix(self,edges_matrix):
        self.edges_matrix = edges_matrix
    def Fd_caculate(self):
        for index in  self.edges_matrix.columns:
            for col in  self.edges_matrix.columns:
                if self.edges_matrix[col][index]>=self.fuse_num:
                     self.edges_matrix.at[col,index] = 1
                else :
                     self.edges_matrix.at[col,index] = 0
        fuse_edges = to_dag(edges_matrix)
        fuseDAG = DAG()
        fuseDAG.add_edges_from(fuse_edges)
        return  fuseDAG

In [23]:
DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/tenSiteDTDisc.csv").drop(columns="Unnamed: 0",axis = 1)
DB_1["label"].value_counts()

0    13660
1     5781
Name: label, dtype: int64

In [24]:
DB_1 = DB_1.astype(np.int16)

In [34]:
# 数据分割
hos_id = [122, 141, 142, 227, 243, 382, 390, 403, 420, 435]
client_data = []
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["creatinine","patientunitstayid","hospitalid"],axis = 1))
    
feature = client_data[0].columns

In [35]:
black_list = []
for i in feature:
    if i != "label":
        black_list.append(("label",i))

In [36]:
from pgmpy.estimators import K2Score

In [ ]:

# 结构训练
train_round = 10
round_num = 0
client_num = 10
clients = []
for i in range(client_num):
# 创建客户端，并且传入数据和结构学习模型
    clients.append(client(client_data[i],i,[]))
# 初始化服务器
sever_1 = sever(5)
edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
while(round_num<train_round):
    #利用二维矩阵传递边结构信息
    
    #开始训练
    #重置矩阵
    edges_matrix.iloc[:,:] = 0
    #第一轮，先本地客户端建立初始模型
    if round_num == 0:
        edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        print("初始化训练")
        for i in range(client_num):
            clients[i].fit()
    
    #开始聚集客户端边的信息
    for i in range(client_num):
        #将客户端最好的模型边信息转换为矩阵形式
        edges_matrix+=to_matrix(clients[i].get_model().edges(),feature)
    #服务器开始汇集信息，并且将矩阵转换为Dag
    
    sever_1.get_edges_matrix(edges_matrix)
    fuseDAG = sever_1.Fd_caculate()
    print("第",train_round,"此迭代")
    print_dag(fuseDAG)
    #更新客户端模型
    for i in range(client_num):
        clients[i]. update_model(fuseDAG)
    round_num+=1

初始化训练


  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
for i in to_dag(sever_1.edges_matrix):
    print(i[0],"->",i[1],";")

In [ ]:
import rpy2.robjects as robjects

In [ ]:
#‘pi’为R的内置变量
# 第一种
t0=robjects.r['pi']
print(t0[0])
# 第二种
t1=robjects.r('pi')# 这种方法从某种程度上讲是万能的，因为可以将任意大小和长度的R代码写成一个python字符串，之后通过robjects.r('Rcode')调用执行
print(t1[0])
a = robjects.r('a<-c(1,2,3)')
print(a)
#第三种
t2=robjects.r.pi# 这种方法对于名称中有“点号”的变量会出问题，比如data.frame/read.csv等
print(t2[0])